In [15]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm

# Paso a): Descargar los datos
url = "https://raw.githubusercontent.com/asegura4488/Database/main/MetodosComputacionalesReforma/Gaussiano.csv"
data = pd.read_csv(url)
x = data["x"].values

# Paso b): Distribución a priori uniforme
def prior(mu, sigma):
    if 3 <= mu <= 5 and 0.5 <= sigma <= 3.5:
        return 1
    else:
        return 0

# Paso c): Función de likelihood gaussiana
def likelihood(x, mu, sigma):
    N = len(x)
    product_likelihood = 1
    for xi in x:
        likelihood = (1 / (sigma * np.sqrt(2 * np.pi))) * np.exp(-(xi - mu)**2 / (2 * sigma**2))
        product_likelihood *= likelihood
    return product_likelihood

# Paso d): Cálculo del logaritmo de la distribución posterior
def log_posterior(mu, sigma, x):
    return np.log(likelihood(x, mu, sigma) * prior(mu, sigma))

# Paso e): Algoritmo de Metrópolis-Hastings
def metropolis_hastings(x, initial_mu, initial_sigma, num_steps, delta=0.5):
    mu = np.zeros(num_steps)
    sigma = np.zeros(num_steps)
    mu[0] = initial_mu
    sigma[0] = initial_sigma

    for i in range(0, num_steps-1):
        # Propuesta de nuevo valor para mu y sigma
        new_mu = mu[i] + np.random.uniform(-delta, delta)
        new_sigma = sigma[i] + np.random.uniform(-delta, delta)

        # Calcula el logaritmo de la probabilidad de aceptación
        log_acceptance = log_posterior(new_mu, new_sigma, x) - log_posterior(mu[i], sigma[i], x)

        # Genera un número aleatorio entre 0 y 1
        acceptance_prob = np.random.rand()

        # Acepta o rechaza la propuesta basado en el logaritmo de la probabilidad de aceptación
        if np.log(acceptance_prob) < log_acceptance:
            mu[i+1] = new_mu
            sigma[i+1] = new_sigma

        else:
            mu[i+1] = mu[i]
            sigma[i+1] = sigma[i]

    return mu, sigma

# Paso f): Estimación de los parámetros mu y sigma
num_steps = 2 * 10**4
initial_mu = 4
initial_sigma = 2
mu_samples, sigma_samples = metropolis_hastings(x, initial_mu, initial_sigma, num_steps)

best_mu = np.mean(mu_samples)
best_sigma = np.mean(sigma_samples)
print("Mejor valor del parámetro mu:", best_mu)
print("Mejor valor del parámetro sigma:", best_sigma)

# Paso g): Calcular los errores de los parámetros en un intervalo de confianza del 68%
liminfmu = np.percentile(mu_samples,16)
limsummu = np.percentile(mu_samples,84)
liminfsigma = np.percentile(sigma_samples,16)
limsumsigma = np.percentile(sigma_samples,84)
confidence = 68

conclusionmu = "µ estimado = {:.2f} ± {:.2f} a un nivel de confianza del {}%"
conclusionsigma = "σ estimado = {:.2f} ± {:.2f} a un nivel de confianza del {}%"

print(conclusionmu.format(best_mu, (limsummu - best_mu), confidence))
print(conclusionsigma.format(best_sigma, (limsumsigma - best_sigma), confidence))


<ipython-input-15-03d936be906b>:29: RuntimeWarning: divide by zero encountered in log
  return np.log(likelihood(x, mu, sigma) * prior(mu, sigma))


Mejor valor del parámetro mu: 4.017910676645029
Mejor valor del parámetro sigma: 1.8375720893799452
µ estimado = 4.02 ± 0.19 a un nivel de confianza del 68%
σ estimado = 1.84 ± 0.13 a un nivel de confianza del 68%
